In [ ]:
%load_ext Cython

In [ ]:
import numpy as np

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# Code

In [ ]:
def boundary_set(mask):
    ''' List pixels that are on the boundary of the mask '''
    n = mask.shape[0]
    i1 = np.arange(n)
    j1 = np.arange(n)
    j2,i2 = np.meshgrid(i1,j1)

    dx,dy = np.gradient(mask)
    boundary_map = (~mask) & (dx*dx + dy*dy != 0)

    bi = i2[boundary_map]
    bj = j2[boundary_map]

    return bi,bj

In [ ]:
def bounding_box(boundary):
    ''' Build a bounding box for interior pixels '''

    n = boundary.shape[0]
    assert n == boundary.shape[1]
    assert np.all((boundary == 0) | (boundary == 1))
    imin = n+1
    imax = -1
    jmin = n+1
    jmax = -1
    iproj = np.product(boundary,axis=1)
    jproj = np.product(boundary,axis=0)
    for i in range(n):
        if not iproj[i]:
            if i < imin:
                imin = i
            if i > imax:
                imax = i
        if not jproj[i]:
            if i < jmin:
                jmin = i
            if i > jmax:
                jmax = i
    
    imin = max(0,imin-1)
    jmin = max(0,jmin-1)
    imax = min(n-1,imax+1)
    jmax = min(n-1,jmax+1)
    return imin,jmin,imax,jmax

In [ ]:
def boundary_distance(mask):
    '''
    Find distance of points in mask to points on boundary of mask
    The post office problem
    '''
    bi,bj = boundary_set(mask)
    nb = len(bi)

    imin,jmin,imax,jmax = bounding_box(mask)

    c_code = '''
    int b,i,j,i0,j0;
    int d,di,dj;
    for(b=0;b<nb;b++) {
        i0 = bi(b);
        j0 = bj(b);
        if(i0 < imin) continue;
        if(j0 < jmin) continue;
        if(i0 > imax) continue;
        if(j0 > jmax) continue;
        for(i=imin;i<imax;i++) {
            for(j=jmin;j<jmax;j++) {
                if(mask(i,j)) {
                    di = i - i0;
                    dj = j - j0;
                    d = di*di + dj*dj;
                    if(d < distance(i,j)) {
                        distance(i,j) = d;
                    }
                }
            }
        }
    }
    '''

    n = mask.shape[0]
    distance = np.zeros((n,n),dtype=np.int32) + 10*n*n
    vars = 'nb n bi bj mask distance imin imax jmin jmax'.split()
    weave.inline(c_code,vars,type_converters=weave.converters.blitz)

    distance = np.sqrt(distance)
    distance[~mask] = 0

    return distance

# Fake data

In [ ]:
def circular_mask(n, r):
    mask = np.zeros((n, n),dtype=bool)
    for i in range(n):
        for j in range(n):
            if (i - n / 2)**2 + (j - n / 2)**2 < r**2:
                mask[i][j]=True
    return mask

# New code

In [ ]:
%%cython --annotate

import numpy as np
cimport numpy as np

cimport cython

from libc.stdlib cimport malloc, calloc, free

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef get_start(np.ndarray[np.uint8_t, cast=True, ndim=2] mask):
    m = mask.shape[0]
    n = mask.shape[1]
    for i in range(m):
        for j in range(n):
            if mask[i, j]:
                return complex(i, j)



class Turtle(object):
    def __init__(self, start):
        self.loc = start
        self.direction = 1j
    def walk(self):
        self.loc += self.direction
    def turn_left(self):
        self.direction *= -1j
    def turn_right(self):
        self.direction *= 1j
    def get_loc(self):
        return self.loc


@cython.boundscheck(False)
@cython.wraparound(False)
def get_boundary(mask, debug=False, stop=10):
    start = get_start(mask)
    boundary_list = [start]
    boundary_set = {start}
    turtle = Turtle(start)
    if debug:
        counter = 0
    while True:
        if debug:
            print(turtle.get_loc())
        loc = turtle.get_loc()
        if mask[int(loc.real), int(loc.imag)]:
            turtle.turn_left()
        else:
            turtle.turn_right()
        turtle.walk()
        loc = turtle.get_loc()
        if mask[int(loc.real), int(loc.imag)]:
            if loc not in boundary_set:
                boundary_list.append(loc)
                boundary_set.add(loc)
        if debug:
            counter += 1
            if counter == stop:
                break
        if loc == start:
            break
    return boundary_list

In [ ]:
n = 6
r = 2
mask = circular_mask(n, r)
boundary = get_boundary(mask, debug=False)

In [ ]:
assert boundary == [(2+2j), (3+2j), (4+2j), (4+3j), (4+4j), (3+4j), (2+4j), (2+3j)]